In [1]:
using Flux,Random,Optimisers,Statistics, Optimisers

In [2]:
mutable struct GlobalModel
    subnets
end
function GlobalModel()
    model1=Chain(Dense(3=>3,bias=false,init=rand))
    model2=Chain(Dense(3=>1,bias=false,init=rand))
    subnets=[model1,model2]
    GlobalModel(subnets)
end
function call_train(glob::GlobalModel,inputs)
    return glob.subnets[2](glob.subnets[1](inputs[1]))
end

(glob::GlobalModel)(inputs) =call_train(glob,inputs)
Flux.@functor GlobalModel 

In [3]:
glob=GlobalModel();

In [4]:
inputs=[randn((3,5)),randn((2,5))];

In [5]:
glob(inputs)

1×5 Matrix{Float64}:
 1.43063  1.30343  0.407766  -0.288406  -0.716602

In [6]:
inputs[1]

3×5 Matrix{Float64}:
  1.86263   0.495549   0.861905   0.00699441  -1.34859
  0.818025  0.443501  -1.17708   -2.31773     -0.0289284
 -0.628183  1.17661    0.425016   1.20513      0.566821

In [7]:
sum(inputs[1],dims=1)

1×5 Matrix{Float64}:
 2.05247  2.11566  0.109837  -1.1056  -0.810699

In [8]:
function loss(y_terminal,inputs)
    delta = (y_terminal.-sum(inputs[1],dims=1)).^2
    return mean(delta)
end

function train!(glob::GlobalModel)
    #opt = Optimisers.Adam(0.01)
    #opt_state = Optimisers.setup(opt, glob)
    optim = Flux.setup(Flux.Adam(0.001), glob)
    
    my_log = []
    for epoch in 1:2000
        input=[randn((3,5)),randn((2,5))]
        grad = gradient(glob) do m
        result = m(input)
        loss(result, input)
        end

        if epoch%500==0
            inp=[randn((3,5)),randn((2,5))]
            println("Epoch ",epoch, " losses ",loss(glob(inp),inp))
        end
        Flux.update!(optim, glob, grad[1])
        #println(Flux.params(glob))
    end
end

train! (generic function with 1 method)

In [10]:
train!(glob)

Epoch 500 losses 6.844827402643467e-8
Epoch 1000 losses 1.5837124566712424e-18
Epoch 1500 losses 1.8925103784014487e-19
Epoch 2000 losses 5.147682197588752e-12


In [11]:
glob(inputs)

1×5 Matrix{Float64}:
 2.05247  2.11566  0.109829  -1.10562  -0.810697